In [1]:
import requests
import json
import tkinter as tk
from tkinter import ttk, scrolledtext
import threading
import speech_recognition as sr
import pyttsx3
from datetime import datetime
import queue
import re

class InteractiveChatBot:
    def __init__(self):
        # API Configuration
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "YOUR_SITE_URL",
            "X-Title": "Interactive ChatBot"
        }
       
        # Conversation history
        self.conversation_history = []
       
        # Voice components
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.tts_engine = pyttsx3.init()
        self.setup_tts()
       
        # GUI components
        self.setup_gui()
       
        # Threading
        self.voice_queue = queue.Queue()
        self.is_listening = False
       
    def setup_tts(self):
        """Configure text-to-speech settings"""
        voices = self.tts_engine.getProperty('voices')
        if voices:
            # Try to set a female voice if available
            for voice in voices:
                if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                    self.tts_engine.setProperty('voice', voice.id)
                    break
       
        self.tts_engine.setProperty('rate', 150)  # Speed
        self.tts_engine.setProperty('volume', 0.8)  # Volume
   
    def setup_gui(self):
        """Create the GUI interface"""
        self.root = tk.Tk()
        self.root.title("Interactive AI ChatBot & VoiceBot")
        self.root.geometry("800x600")
        self.root.configure(bg='#f0f0f0')
       
        # Style configuration
        style = ttk.Style()
        style.theme_use('clam')
       
        # Header Frame
        header_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        header_frame.pack(fill='x', padx=10, pady=(10,0))
        header_frame.pack_propagate(False)
       
        title_label = tk.Label(
            header_frame,
            text="🤖 Interactive AI ChatBot & VoiceBot",
            bg='#2c3e50',
            fg='white',
            font=('Arial', 16, 'bold')
        )
        title_label.pack(expand=True)
       
        # Main container
        main_frame = tk.Frame(self.root, bg='#f0f0f0')
        main_frame.pack(fill='both', expand=True, padx=10, pady=10)
       
        # Chat display area
        self.chat_display = scrolledtext.ScrolledText(
            main_frame,
            wrap=tk.WORD,
            width=70,
            height=20,
            font=('Arial', 10),
            bg='white',
            fg='#333333',
            state=tk.DISABLED
        )
        self.chat_display.pack(fill='both', expand=True, pady=(0, 10))
       
        # Input frame
        input_frame = tk.Frame(main_frame, bg='#f0f0f0')
        input_frame.pack(fill='x', pady=(0, 10))
       
        # Text input
        self.text_input = tk.Entry(
            input_frame,
            font=('Arial', 11),
            bg='white',
            fg='#333333'
        )
        self.text_input.pack(side='left', fill='x', expand=True, padx=(0, 10))
        self.text_input.bind('<Return>', self.send_text_message)
       
        # Send button
        send_btn = tk.Button(
            input_frame,
            text="Send 📤",
            command=self.send_text_message,
            bg='#3498db',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        send_btn.pack(side='right')
       
        # Control buttons frame
        control_frame = tk.Frame(main_frame, bg='#f0f0f0')
        control_frame.pack(fill='x')
       
        # Voice button
        self.voice_btn = tk.Button(
            control_frame,
            text="🎤 Start Voice",
            command=self.toggle_voice,
            bg='#e74c3c',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        self.voice_btn.pack(side='left', padx=(0, 10))
       
        # Clear chat button
        clear_btn = tk.Button(
            control_frame,
            text="🗑️ Clear Chat",
            command=self.clear_chat,
            bg='#95a5a6',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        clear_btn.pack(side='left', padx=(0, 10))
       
        # Status label
        self.status_label = tk.Label(
            control_frame,
            text="Ready to chat!",
            bg='#f0f0f0',
            fg='#27ae60',
            font=('Arial', 10, 'bold')
        )
        self.status_label.pack(side='right')
       
        # Initial welcome message
        self.add_message("Bot", "Hello! I'm your interactive AI assistant. You can type messages or use voice input. How can I help you today?", "#3498db")
   
    def add_message(self, sender, message, color="#333333"):
        """Add a message to the chat display"""
        self.chat_display.config(state=tk.NORMAL)
       
        timestamp = datetime.now().strftime("%H:%M:%S")
       
        # Add sender and timestamp
        self.chat_display.insert(tk.END, f"\n[{timestamp}] {sender}: ", f"sender_{sender}")
        self.chat_display.insert(tk.END, f"{message}\n", "message")
       
        # Configure tags for styling
        self.chat_display.tag_config(f"sender_{sender}", foreground=color, font=('Arial', 10, 'bold'))
        self.chat_display.tag_config("message", foreground="#333333", font=('Arial', 10))
       
        self.chat_display.config(state=tk.DISABLED)
        self.chat_display.see(tk.END)
   
    def get_ai_response(self, user_message):
        """Get response from AI API"""
        try:
            # Add user message to conversation history
            self.conversation_history.append({"role": "user", "content": user_message})
           
            # Keep only last 10 messages to avoid token limits
            if len(self.conversation_history) > 10:
                self.conversation_history = self.conversation_history[-10:]
           
            # Prepare API request
            data = {
                "model": "deepseek/deepseek-r1:free",
                "messages": [
                    {"role": "system", "content": "You are a helpful, friendly, and conversational AI assistant. Provide clear, engaging responses and feel free to ask follow-up questions to keep the conversation flowing."},
                    *self.conversation_history
                ],
                "temperature": 0.7,
                "max_tokens": 500
            }
           
            response = requests.post(
                self.api_url,
                headers=self.headers,
                data=json.dumps(data),
                timeout=30
            )
           
            if response.status_code == 200:
                response_data = response.json()
                ai_message = response_data['choices'][0]['message']['content']
               
                # Add AI response to conversation history
                self.conversation_history.append({"role": "assistant", "content": ai_message})
               
                return ai_message
            else:
                return f"Sorry, I encountered an error: {response.status_code}"
               
        except requests.exceptions.Timeout:
            return "Sorry, the request timed out. Please try again."
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
   
    def send_text_message(self, event=None):
        """Send text message"""
        message = self.text_input.get().strip()
        if not message:
            return
       
        # Add user message to display
        self.add_message("You", message, "#27ae60")
        self.text_input.delete(0, tk.END)
       
        # Update status
        self.status_label.config(text="Getting response...", fg="#f39c12")
        self.root.update()
       
        # Get AI response in a separate thread
        def get_response():
            ai_response = self.get_ai_response(message)
            self.root.after(0, lambda: self.handle_ai_response(ai_response))
       
        threading.Thread(target=get_response, daemon=True).start()
   
    def handle_ai_response(self, response):
        """Handle AI response and speak it"""
        self.add_message("Bot", response, "#3498db")
        self.status_label.config(text="Ready to chat!", fg="#27ae60")
       
        # Speak the response
        self.speak_text(response)
   
    def speak_text(self, text):
        """Convert text to speech"""
        def speak():
            try:
                # Clean text for better speech
                clean_text = re.sub(r'[*_`#]', '', text)  # Remove markdown
                clean_text = re.sub(r'\n+', '. ', clean_text)  # Replace newlines with periods
                self.tts_engine.say(clean_text)
                self.tts_engine.runAndWait()
            except:
                pass
       
        threading.Thread(target=speak, daemon=True).start()
   
    def toggle_voice(self):
        """Toggle voice input"""
        if not self.is_listening:
            self.start_voice_input()
        else:
            self.stop_voice_input()
   
    def start_voice_input(self):
        """Start voice input"""
        self.is_listening = True
        self.voice_btn.config(text="🔴 Stop Voice", bg="#e74c3c")
        self.status_label.config(text="Listening... Speak now!", fg="#e74c3c")
       
        def listen():
            try:
                with self.microphone as source:
                    self.recognizer.adjust_for_ambient_noise(source)
               
                while self.is_listening:
                    try:
                        with self.microphone as source:
                            audio = self.recognizer.listen(source, timeout=1, phrase_time_limit=5)
                       
                        text = self.recognizer.recognize_google(audio)
                        if text:
                            self.root.after(0, lambda: self.handle_voice_input(text))
                    except sr.WaitTimeoutError:
                        continue
                    except sr.UnknownValueError:
                        continue
                    except Exception as e:
                        self.root.after(0, lambda: self.status_label.config(text=f"Voice error: {str(e)}", fg="#e74c3c"))
                        break
            except Exception as e:
                self.root.after(0, lambda: self.status_label.config(text=f"Microphone error: {str(e)}", fg="#e74c3c"))
       
        threading.Thread(target=listen, daemon=True).start()
   
    def stop_voice_input(self):
        """Stop voice input"""
        self.is_listening = False
        self.voice_btn.config(text="🎤 Start Voice", bg="#27ae60")
        self.status_label.config(text="Voice input stopped", fg="#95a5a6")
   
    def handle_voice_input(self, text):
        """Handle voice input"""
        self.text_input.delete(0, tk.END)
        self.text_input.insert(0, text)
        self.send_text_message()
   
    def clear_chat(self):
        """Clear chat history"""
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.delete(1.0, tk.END)
        self.chat_display.config(state=tk.DISABLED)
        self.conversation_history.clear()
        self.add_message("Bot", "Chat cleared! How can I help you?", "#3498db")
   
    def run(self):
        """Start the application"""
        try:
            self.root.mainloop()
        except KeyboardInterrupt:
            self.stop_voice_input()

# Simple command-line version
class SimpleChatBot:
    def __init__(self):
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        self.conversation_history = []
   
    def get_response(self, message):
        """Get AI response"""
        self.conversation_history.append({"role": "user", "content": message})
       
        data = {
            "model": "deepseek/deepseek-r1:free",
            "messages": [
                {"role": "system", "content": "You are a helpful AI assistant."},
                *self.conversation_history[-10:]  # Keep last 10 messages
            ]
        }
       
        try:
            response = requests.post(self.api_url, headers=self.headers, data=json.dumps(data))
            if response.status_code == 200:
                ai_message = response.json()['choices'][0]['message']['content']
                self.conversation_history.append({"role": "assistant", "content": ai_message})
                return ai_message
            else:
                return f"Error: {response.status_code}"
        except Exception as e:
            return f"Error: {str(e)}"
   
    def run(self):
        """Run simple chat"""
        print("🤖 Simple ChatBot started! Type 'quit' to exit.")
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() in ['quit', 'exit']:
                break
            if user_input:
                response = self.get_response(user_input)
                print(f"Bot: {response}")

if __name__ == "__main__":
    print("Choose mode:")
    print("1. GUI ChatBot with Voice (Recommended)")
    print("2. Simple Command Line ChatBot")
   
    choice = input("Enter choice (1 or 2): ").strip()
   
    if choice == "2":
        bot = SimpleChatBot()
        bot.run()
    else:
        try:
            bot = InteractiveChatBot()
            bot.run()
        except ImportError as e:
            print(f"GUI dependencies missing: {e}")
            print("Installing required packages:")
            print("pip install speechrecognition pyttsx3 pyaudio")
            print("\nRunning simple version instead...")
            bot = SimpleChatBot()
            bot.run()

ModuleNotFoundError: No module named 'speech_recognition'

In [3]:
!pip install SpeechRecognition


   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/32.9 MB 1.1 MB/s eta 0:00:32
   ---------------------------------------- 0.3/32.9 MB 2.3 MB/s eta 0:00:15
    --------------------------------------- 0.6/32.9 MB 3.2 MB/s eta 0:00:11
   - -------------------------------------- 1.2/32.9 MB 5.2 MB/s eta 0:00:07
   -- ------------------------------------- 2.1/32.9 MB 7.4 MB/s eta 0:00:05
   --- ------------------------------------ 3.2/32.9 MB 9.3 MB/s eta 0:00:04
   ----- ---------------------------------- 4.4/32.9 MB 10.4 MB/s eta 0:00:03
   ------- -------------------------------- 5.8/32.9 MB 12.0 MB/s eta 0:00:03
   --------- ------------------------------ 7.8/32.9 MB 14.2 MB/s eta 0:00:02
   ---------- ----------------------------- 8.4/32.9 MB 13.4 MB/s eta 0:00:02
   ----------- ---------------------------- 9.4/32.9 MB 14.0 MB/s eta 0:00:02
   -----

In [4]:
import requests
import json
import tkinter as tk
from tkinter import ttk, scrolledtext
import threading
import speech_recognition as sr
import pyttsx3
from datetime import datetime
import queue
import re

class InteractiveChatBot:
    def __init__(self):
        # API Configuration
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "YOUR_SITE_URL",
            "X-Title": "Interactive ChatBot"
        }
       
        # Conversation history
        self.conversation_history = []
       
        # Voice components
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.tts_engine = pyttsx3.init()
        self.setup_tts()
       
        # GUI components
        self.setup_gui()
       
        # Threading
        self.voice_queue = queue.Queue()
        self.is_listening = False
       
    def setup_tts(self):
        """Configure text-to-speech settings"""
        voices = self.tts_engine.getProperty('voices')
        if voices:
            # Try to set a female voice if available
            for voice in voices:
                if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                    self.tts_engine.setProperty('voice', voice.id)
                    break
       
        self.tts_engine.setProperty('rate', 150)  # Speed
        self.tts_engine.setProperty('volume', 0.8)  # Volume
   
    def setup_gui(self):
        """Create the GUI interface"""
        self.root = tk.Tk()
        self.root.title("Interactive AI ChatBot & VoiceBot")
        self.root.geometry("800x600")
        self.root.configure(bg='#f0f0f0')
       
        # Style configuration
        style = ttk.Style()
        style.theme_use('clam')
       
        # Header Frame
        header_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        header_frame.pack(fill='x', padx=10, pady=(10,0))
        header_frame.pack_propagate(False)
       
        title_label = tk.Label(
            header_frame,
            text="🤖 Interactive AI ChatBot & VoiceBot",
            bg='#2c3e50',
            fg='white',
            font=('Arial', 16, 'bold')
        )
        title_label.pack(expand=True)
       
        # Main container
        main_frame = tk.Frame(self.root, bg='#f0f0f0')
        main_frame.pack(fill='both', expand=True, padx=10, pady=10)
       
        # Chat display area
        self.chat_display = scrolledtext.ScrolledText(
            main_frame,
            wrap=tk.WORD,
            width=70,
            height=20,
            font=('Arial', 10),
            bg='white',
            fg='#333333',
            state=tk.DISABLED
        )
        self.chat_display.pack(fill='both', expand=True, pady=(0, 10))
       
        # Input frame
        input_frame = tk.Frame(main_frame, bg='#f0f0f0')
        input_frame.pack(fill='x', pady=(0, 10))
       
        # Text input
        self.text_input = tk.Entry(
            input_frame,
            font=('Arial', 11),
            bg='white',
            fg='#333333'
        )
        self.text_input.pack(side='left', fill='x', expand=True, padx=(0, 10))
        self.text_input.bind('<Return>', self.send_text_message)
       
        # Send button
        send_btn = tk.Button(
            input_frame,
            text="Send 📤",
            command=self.send_text_message,
            bg='#3498db',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        send_btn.pack(side='right')
       
        # Control buttons frame
        control_frame = tk.Frame(main_frame, bg='#f0f0f0')
        control_frame.pack(fill='x')
       
        # Voice button
        self.voice_btn = tk.Button(
            control_frame,
            text="🎤 Start Voice",
            command=self.toggle_voice,
            bg='#e74c3c',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        self.voice_btn.pack(side='left', padx=(0, 10))
       
        # Clear chat button
        clear_btn = tk.Button(
            control_frame,
            text="🗑️ Clear Chat",
            command=self.clear_chat,
            bg='#95a5a6',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        clear_btn.pack(side='left', padx=(0, 10))
       
        # Status label
        self.status_label = tk.Label(
            control_frame,
            text="Ready to chat!",
            bg='#f0f0f0',
            fg='#27ae60',
            font=('Arial', 10, 'bold')
        )
        self.status_label.pack(side='right')
       
        # Initial welcome message
        self.add_message("Bot", "Hello! I'm your interactive AI assistant. You can type messages or use voice input. How can I help you today?", "#3498db")
   
    def add_message(self, sender, message, color="#333333"):
        """Add a message to the chat display"""
        self.chat_display.config(state=tk.NORMAL)
       
        timestamp = datetime.now().strftime("%H:%M:%S")
       
        # Add sender and timestamp
        self.chat_display.insert(tk.END, f"\n[{timestamp}] {sender}: ", f"sender_{sender}")
        self.chat_display.insert(tk.END, f"{message}\n", "message")
       
        # Configure tags for styling
        self.chat_display.tag_config(f"sender_{sender}", foreground=color, font=('Arial', 10, 'bold'))
        self.chat_display.tag_config("message", foreground="#333333", font=('Arial', 10))
       
        self.chat_display.config(state=tk.DISABLED)
        self.chat_display.see(tk.END)
   
    def get_ai_response(self, user_message):
        """Get response from AI API"""
        try:
            # Add user message to conversation history
            self.conversation_history.append({"role": "user", "content": user_message})
           
            # Keep only last 10 messages to avoid token limits
            if len(self.conversation_history) > 10:
                self.conversation_history = self.conversation_history[-10:]
           
            # Prepare API request
            data = {
                "model": "deepseek/deepseek-r1:free",
                "messages": [
                    {"role": "system", "content": "You are a helpful, friendly, and conversational AI assistant. Provide clear, engaging responses and feel free to ask follow-up questions to keep the conversation flowing."},
                    *self.conversation_history
                ],
                "temperature": 0.7,
                "max_tokens": 500
            }
           
            response = requests.post(
                self.api_url,
                headers=self.headers,
                data=json.dumps(data),
                timeout=30
            )
           
            if response.status_code == 200:
                response_data = response.json()
                ai_message = response_data['choices'][0]['message']['content']
               
                # Add AI response to conversation history
                self.conversation_history.append({"role": "assistant", "content": ai_message})
               
                return ai_message
            else:
                return f"Sorry, I encountered an error: {response.status_code}"
               
        except requests.exceptions.Timeout:
            return "Sorry, the request timed out. Please try again."
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
   
    def send_text_message(self, event=None):
        """Send text message"""
        message = self.text_input.get().strip()
        if not message:
            return
       
        # Add user message to display
        self.add_message("You", message, "#27ae60")
        self.text_input.delete(0, tk.END)
       
        # Update status
        self.status_label.config(text="Getting response...", fg="#f39c12")
        self.root.update()
       
        # Get AI response in a separate thread
        def get_response():
            ai_response = self.get_ai_response(message)
            self.root.after(0, lambda: self.handle_ai_response(ai_response))
       
        threading.Thread(target=get_response, daemon=True).start()
   
    def handle_ai_response(self, response):
        """Handle AI response and speak it"""
        self.add_message("Bot", response, "#3498db")
        self.status_label.config(text="Ready to chat!", fg="#27ae60")
       
        # Speak the response
        self.speak_text(response)
   
    def speak_text(self, text):
        """Convert text to speech"""
        def speak():
            try:
                # Clean text for better speech
                clean_text = re.sub(r'[*_`#]', '', text)  # Remove markdown
                clean_text = re.sub(r'\n+', '. ', clean_text)  # Replace newlines with periods
                self.tts_engine.say(clean_text)
                self.tts_engine.runAndWait()
            except:
                pass
       
        threading.Thread(target=speak, daemon=True).start()
   
    def toggle_voice(self):
        """Toggle voice input"""
        if not self.is_listening:
            self.start_voice_input()
        else:
            self.stop_voice_input()
   
    def start_voice_input(self):
        """Start voice input"""
        self.is_listening = True
        self.voice_btn.config(text="🔴 Stop Voice", bg="#e74c3c")
        self.status_label.config(text="Listening... Speak now!", fg="#e74c3c")
       
        def listen():
            try:
                with self.microphone as source:
                    self.recognizer.adjust_for_ambient_noise(source)
               
                while self.is_listening:
                    try:
                        with self.microphone as source:
                            audio = self.recognizer.listen(source, timeout=1, phrase_time_limit=5)
                       
                        text = self.recognizer.recognize_google(audio)
                        if text:
                            self.root.after(0, lambda: self.handle_voice_input(text))
                    except sr.WaitTimeoutError:
                        continue
                    except sr.UnknownValueError:
                        continue
                    except Exception as e:
                        self.root.after(0, lambda: self.status_label.config(text=f"Voice error: {str(e)}", fg="#e74c3c"))
                        break
            except Exception as e:
                self.root.after(0, lambda: self.status_label.config(text=f"Microphone error: {str(e)}", fg="#e74c3c"))
       
        threading.Thread(target=listen, daemon=True).start()
   
    def stop_voice_input(self):
        """Stop voice input"""
        self.is_listening = False
        self.voice_btn.config(text="🎤 Start Voice", bg="#27ae60")
        self.status_label.config(text="Voice input stopped", fg="#95a5a6")
   
    def handle_voice_input(self, text):
        """Handle voice input"""
        self.text_input.delete(0, tk.END)
        self.text_input.insert(0, text)
        self.send_text_message()
   
    def clear_chat(self):
        """Clear chat history"""
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.delete(1.0, tk.END)
        self.chat_display.config(state=tk.DISABLED)
        self.conversation_history.clear()
        self.add_message("Bot", "Chat cleared! How can I help you?", "#3498db")
   
    def run(self):
        """Start the application"""
        try:
            self.root.mainloop()
        except KeyboardInterrupt:
            self.stop_voice_input()

# Simple command-line version
class SimpleChatBot:
    def __init__(self):
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        self.conversation_history = []
   
    def get_response(self, message):
        """Get AI response"""
        self.conversation_history.append({"role": "user", "content": message})
       
        data = {
            "model": "deepseek/deepseek-r1:free",
            "messages": [
                {"role": "system", "content": "You are a helpful AI assistant."},
                *self.conversation_history[-10:]  # Keep last 10 messages
            ]
        }
       
        try:
            response = requests.post(self.api_url, headers=self.headers, data=json.dumps(data))
            if response.status_code == 200:
                ai_message = response.json()['choices'][0]['message']['content']
                self.conversation_history.append({"role": "assistant", "content": ai_message})
                return ai_message
            else:
                return f"Error: {response.status_code}"
        except Exception as e:
            return f"Error: {str(e)}"
   
    def run(self):
        """Run simple chat"""
        print("🤖 Simple ChatBot started! Type 'quit' to exit.")
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() in ['quit', 'exit']:
                break
            if user_input:
                response = self.get_response(user_input)
                print(f"Bot: {response}")

if __name__ == "__main__":
    print("Choose mode:")
    print("1. GUI ChatBot with Voice (Recommended)")
    print("2. Simple Command Line ChatBot")
   
    choice = input("Enter choice (1 or 2): ").strip()
   
    if choice == "2":
        bot = SimpleChatBot()
        bot.run()
    else:
        try:
            bot = InteractiveChatBot()
            bot.run()
        except ImportError as e:
            print(f"GUI dependencies missing: {e}")
            print("Installing required packages:")
            print("pip install speechrecognition pyttsx3 pyaudio")
            print("\nRunning simple version instead...")
            bot = SimpleChatBot()
            bot.run()

ModuleNotFoundError: No module named 'pyttsx3'

In [6]:
!pip install pyttsx3

In [7]:
import requests
import json
import tkinter as tk
from tkinter import ttk, scrolledtext
import threading
import speech_recognition as sr
import pyttsx3
from datetime import datetime
import queue
import re

class InteractiveChatBot:
    def __init__(self):
        # API Configuration
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "YOUR_SITE_URL",
            "X-Title": "Interactive ChatBot"
        }
       
        # Conversation history
        self.conversation_history = []
       
        # Voice components
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.tts_engine = pyttsx3.init()
        self.setup_tts()
       
        # GUI components
        self.setup_gui()
       
        # Threading
        self.voice_queue = queue.Queue()
        self.is_listening = False
       
    def setup_tts(self):
        """Configure text-to-speech settings"""
        voices = self.tts_engine.getProperty('voices')
        if voices:
            # Try to set a female voice if available
            for voice in voices:
                if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                    self.tts_engine.setProperty('voice', voice.id)
                    break
       
        self.tts_engine.setProperty('rate', 150)  # Speed
        self.tts_engine.setProperty('volume', 0.8)  # Volume
   
    def setup_gui(self):
        """Create the GUI interface"""
        self.root = tk.Tk()
        self.root.title("Interactive AI ChatBot & VoiceBot")
        self.root.geometry("800x600")
        self.root.configure(bg='#f0f0f0')
       
        # Style configuration
        style = ttk.Style()
        style.theme_use('clam')
       
        # Header Frame
        header_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        header_frame.pack(fill='x', padx=10, pady=(10,0))
        header_frame.pack_propagate(False)
       
        title_label = tk.Label(
            header_frame,
            text="🤖 Interactive AI ChatBot & VoiceBot",
            bg='#2c3e50',
            fg='white',
            font=('Arial', 16, 'bold')
        )
        title_label.pack(expand=True)
       
        # Main container
        main_frame = tk.Frame(self.root, bg='#f0f0f0')
        main_frame.pack(fill='both', expand=True, padx=10, pady=10)
       
        # Chat display area
        self.chat_display = scrolledtext.ScrolledText(
            main_frame,
            wrap=tk.WORD,
            width=70,
            height=20,
            font=('Arial', 10),
            bg='white',
            fg='#333333',
            state=tk.DISABLED
        )
        self.chat_display.pack(fill='both', expand=True, pady=(0, 10))
       
        # Input frame
        input_frame = tk.Frame(main_frame, bg='#f0f0f0')
        input_frame.pack(fill='x', pady=(0, 10))
       
        # Text input
        self.text_input = tk.Entry(
            input_frame,
            font=('Arial', 11),
            bg='white',
            fg='#333333'
        )
        self.text_input.pack(side='left', fill='x', expand=True, padx=(0, 10))
        self.text_input.bind('<Return>', self.send_text_message)
       
        # Send button
        send_btn = tk.Button(
            input_frame,
            text="Send 📤",
            command=self.send_text_message,
            bg='#3498db',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        send_btn.pack(side='right')
       
        # Control buttons frame
        control_frame = tk.Frame(main_frame, bg='#f0f0f0')
        control_frame.pack(fill='x')
       
        # Voice button
        self.voice_btn = tk.Button(
            control_frame,
            text="🎤 Start Voice",
            command=self.toggle_voice,
            bg='#e74c3c',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        self.voice_btn.pack(side='left', padx=(0, 10))
       
        # Clear chat button
        clear_btn = tk.Button(
            control_frame,
            text="🗑️ Clear Chat",
            command=self.clear_chat,
            bg='#95a5a6',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        clear_btn.pack(side='left', padx=(0, 10))
       
        # Status label
        self.status_label = tk.Label(
            control_frame,
            text="Ready to chat!",
            bg='#f0f0f0',
            fg='#27ae60',
            font=('Arial', 10, 'bold')
        )
        self.status_label.pack(side='right')
       
        # Initial welcome message
        self.add_message("Bot", "Hello! I'm your interactive AI assistant. You can type messages or use voice input. How can I help you today?", "#3498db")
   
    def add_message(self, sender, message, color="#333333"):
        """Add a message to the chat display"""
        self.chat_display.config(state=tk.NORMAL)
       
        timestamp = datetime.now().strftime("%H:%M:%S")
       
        # Add sender and timestamp
        self.chat_display.insert(tk.END, f"\n[{timestamp}] {sender}: ", f"sender_{sender}")
        self.chat_display.insert(tk.END, f"{message}\n", "message")
       
        # Configure tags for styling
        self.chat_display.tag_config(f"sender_{sender}", foreground=color, font=('Arial', 10, 'bold'))
        self.chat_display.tag_config("message", foreground="#333333", font=('Arial', 10))
       
        self.chat_display.config(state=tk.DISABLED)
        self.chat_display.see(tk.END)
   
    def get_ai_response(self, user_message):
        """Get response from AI API"""
        try:
            # Add user message to conversation history
            self.conversation_history.append({"role": "user", "content": user_message})
           
            # Keep only last 10 messages to avoid token limits
            if len(self.conversation_history) > 10:
                self.conversation_history = self.conversation_history[-10:]
           
            # Prepare API request
            data = {
                "model": "deepseek/deepseek-r1:free",
                "messages": [
                    {"role": "system", "content": "You are a helpful, friendly, and conversational AI assistant. Provide clear, engaging responses and feel free to ask follow-up questions to keep the conversation flowing."},
                    *self.conversation_history
                ],
                "temperature": 0.7,
                "max_tokens": 500
            }
           
            response = requests.post(
                self.api_url,
                headers=self.headers,
                data=json.dumps(data),
                timeout=30
            )
           
            if response.status_code == 200:
                response_data = response.json()
                ai_message = response_data['choices'][0]['message']['content']
               
                # Add AI response to conversation history
                self.conversation_history.append({"role": "assistant", "content": ai_message})
               
                return ai_message
            else:
                return f"Sorry, I encountered an error: {response.status_code}"
               
        except requests.exceptions.Timeout:
            return "Sorry, the request timed out. Please try again."
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
   
    def send_text_message(self, event=None):
        """Send text message"""
        message = self.text_input.get().strip()
        if not message:
            return
       
        # Add user message to display
        self.add_message("You", message, "#27ae60")
        self.text_input.delete(0, tk.END)
       
        # Update status
        self.status_label.config(text="Getting response...", fg="#f39c12")
        self.root.update()
       
        # Get AI response in a separate thread
        def get_response():
            ai_response = self.get_ai_response(message)
            self.root.after(0, lambda: self.handle_ai_response(ai_response))
       
        threading.Thread(target=get_response, daemon=True).start()
   
    def handle_ai_response(self, response):
        """Handle AI response and speak it"""
        self.add_message("Bot", response, "#3498db")
        self.status_label.config(text="Ready to chat!", fg="#27ae60")
       
        # Speak the response
        self.speak_text(response)
   
    def speak_text(self, text):
        """Convert text to speech"""
        def speak():
            try:
                # Clean text for better speech
                clean_text = re.sub(r'[*_`#]', '', text)  # Remove markdown
                clean_text = re.sub(r'\n+', '. ', clean_text)  # Replace newlines with periods
                self.tts_engine.say(clean_text)
                self.tts_engine.runAndWait()
            except:
                pass
       
        threading.Thread(target=speak, daemon=True).start()
   
    def toggle_voice(self):
        """Toggle voice input"""
        if not self.is_listening:
            self.start_voice_input()
        else:
            self.stop_voice_input()
   
    def start_voice_input(self):
        """Start voice input"""
        self.is_listening = True
        self.voice_btn.config(text="🔴 Stop Voice", bg="#e74c3c")
        self.status_label.config(text="Listening... Speak now!", fg="#e74c3c")
       
        def listen():
            try:
                with self.microphone as source:
                    self.recognizer.adjust_for_ambient_noise(source)
               
                while self.is_listening:
                    try:
                        with self.microphone as source:
                            audio = self.recognizer.listen(source, timeout=1, phrase_time_limit=5)
                       
                        text = self.recognizer.recognize_google(audio)
                        if text:
                            self.root.after(0, lambda: self.handle_voice_input(text))
                    except sr.WaitTimeoutError:
                        continue
                    except sr.UnknownValueError:
                        continue
                    except Exception as e:
                        self.root.after(0, lambda: self.status_label.config(text=f"Voice error: {str(e)}", fg="#e74c3c"))
                        break
            except Exception as e:
                self.root.after(0, lambda: self.status_label.config(text=f"Microphone error: {str(e)}", fg="#e74c3c"))
       
        threading.Thread(target=listen, daemon=True).start()
   
    def stop_voice_input(self):
        """Stop voice input"""
        self.is_listening = False
        self.voice_btn.config(text="🎤 Start Voice", bg="#27ae60")
        self.status_label.config(text="Voice input stopped", fg="#95a5a6")
   
    def handle_voice_input(self, text):
        """Handle voice input"""
        self.text_input.delete(0, tk.END)
        self.text_input.insert(0, text)
        self.send_text_message()
   
    def clear_chat(self):
        """Clear chat history"""
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.delete(1.0, tk.END)
        self.chat_display.config(state=tk.DISABLED)
        self.conversation_history.clear()
        self.add_message("Bot", "Chat cleared! How can I help you?", "#3498db")
   
    def run(self):
        """Start the application"""
        try:
            self.root.mainloop()
        except KeyboardInterrupt:
            self.stop_voice_input()

# Simple command-line version
class SimpleChatBot:
    def __init__(self):
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        self.conversation_history = []
   
    def get_response(self, message):
        """Get AI response"""
        self.conversation_history.append({"role": "user", "content": message})
       
        data = {
            "model": "deepseek/deepseek-r1:free",
            "messages": [
                {"role": "system", "content": "You are a helpful AI assistant."},
                *self.conversation_history[-10:]  # Keep last 10 messages
            ]
        }
       
        try:
            response = requests.post(self.api_url, headers=self.headers, data=json.dumps(data))
            if response.status_code == 200:
                ai_message = response.json()['choices'][0]['message']['content']
                self.conversation_history.append({"role": "assistant", "content": ai_message})
                return ai_message
            else:
                return f"Error: {response.status_code}"
        except Exception as e:
            return f"Error: {str(e)}"
   
    def run(self):
        """Run simple chat"""
        print("🤖 Simple ChatBot started! Type 'quit' to exit.")
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() in ['quit', 'exit']:
                break
            if user_input:
                response = self.get_response(user_input)
                print(f"Bot: {response}")

if __name__ == "__main__":
    print("Choose mode:")
    print("1. GUI ChatBot with Voice (Recommended)")
    print("2. Simple Command Line ChatBot")
   
    choice = input("Enter choice (1 or 2): ").strip()
   
    if choice == "2":
        bot = SimpleChatBot()
        bot.run()
    else:
        try:
            bot = InteractiveChatBot()
            bot.run()
        except ImportError as e:
            print(f"GUI dependencies missing: {e}")
            print("Installing required packages:")
            print("pip install speechrecognition pyttsx3 pyaudio")
            print("\nRunning simple version instead...")
            bot = SimpleChatBot()
            bot.run()

Choose mode:
1. GUI ChatBot with Voice (Recommended)
2. Simple Command Line ChatBot


Enter choice (1 or 2):  1


AttributeError: Could not find PyAudio; check installation

In [8]:
!pip install pyaudio

   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   ---- ---------------------------------- 20.5/164.1 kB 165.2 kB/s eta 0:00:01
   ------- ------------------------------- 30.7/164.1 kB 220.2 kB/s eta 0:00:01
   -------------- ------------------------ 61.4/164.1 kB 328.2 kB/s eta 0:00:01
   -------------------------------------  163.8/164.1 kB 756.6 kB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 705.6 kB/s eta 0:00:00


In [ ]:
import requests
import json
import tkinter as tk
from tkinter import ttk, scrolledtext
import threading
import speech_recognition as sr
import pyttsx3
from datetime import datetime
import queue
import re

class InteractiveChatBot:
    def __init__(self):
        # API Configuration
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "YOUR_SITE_URL",
            "X-Title": "Interactive ChatBot"
        }
       
        # Conversation history
        self.conversation_history = []
       
        # Voice components
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.tts_engine = pyttsx3.init()
        self.setup_tts()
       
        # GUI components
        self.setup_gui()
       
        # Threading
        self.voice_queue = queue.Queue()
        self.is_listening = False
       
    def setup_tts(self):
        """Configure text-to-speech settings"""
        voices = self.tts_engine.getProperty('voices')
        if voices:
            # Try to set a female voice if available
            for voice in voices:
                if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                    self.tts_engine.setProperty('voice', voice.id)
                    break
       
        self.tts_engine.setProperty('rate', 150)  # Speed
        self.tts_engine.setProperty('volume', 0.8)  # Volume
   
    def setup_gui(self):
        """Create the GUI interface"""
        self.root = tk.Tk()
        self.root.title("Interactive AI ChatBot & VoiceBot")
        self.root.geometry("800x600")
        self.root.configure(bg='#f0f0f0')
       
        # Style configuration
        style = ttk.Style()
        style.theme_use('clam')
       
        # Header Frame
        header_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        header_frame.pack(fill='x', padx=10, pady=(10,0))
        header_frame.pack_propagate(False)
       
        title_label = tk.Label(
            header_frame,
            text="🤖 Interactive AI ChatBot & VoiceBot",
            bg='#2c3e50',
            fg='white',
            font=('Arial', 16, 'bold')
        )
        title_label.pack(expand=True)
       
        # Main container
        main_frame = tk.Frame(self.root, bg='#f0f0f0')
        main_frame.pack(fill='both', expand=True, padx=10, pady=10)
       
        # Chat display area
        self.chat_display = scrolledtext.ScrolledText(
            main_frame,
            wrap=tk.WORD,
            width=70,
            height=20,
            font=('Arial', 10),
            bg='white',
            fg='#333333',
            state=tk.DISABLED
        )
        self.chat_display.pack(fill='both', expand=True, pady=(0, 10))
       
        # Input frame
        input_frame = tk.Frame(main_frame, bg='#f0f0f0')
        input_frame.pack(fill='x', pady=(0, 10))
       
        # Text input
        self.text_input = tk.Entry(
            input_frame,
            font=('Arial', 11),
            bg='white',
            fg='#333333'
        )
        self.text_input.pack(side='left', fill='x', expand=True, padx=(0, 10))
        self.text_input.bind('<Return>', self.send_text_message)
       
        # Send button
        send_btn = tk.Button(
            input_frame,
            text="Send 📤",
            command=self.send_text_message,
            bg='#3498db',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        send_btn.pack(side='right')
       
        # Control buttons frame
        control_frame = tk.Frame(main_frame, bg='#f0f0f0')
        control_frame.pack(fill='x')
       
        # Voice button
        self.voice_btn = tk.Button(
            control_frame,
            text="🎤 Start Voice",
            command=self.toggle_voice,
            bg='#e74c3c',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        self.voice_btn.pack(side='left', padx=(0, 10))
       
        # Clear chat button
        clear_btn = tk.Button(
            control_frame,
            text="🗑️ Clear Chat",
            command=self.clear_chat,
            bg='#95a5a6',
            fg='white',
            font=('Arial', 10, 'bold'),
            relief='flat',
            padx=20
        )
        clear_btn.pack(side='left', padx=(0, 10))
       
        # Status label
        self.status_label = tk.Label(
            control_frame,
            text="Ready to chat!",
            bg='#f0f0f0',
            fg='#27ae60',
            font=('Arial', 10, 'bold')
        )
        self.status_label.pack(side='right')
       
        # Initial welcome message
        self.add_message("Bot", "Hello! I'm your interactive AI assistant. You can type messages or use voice input. How can I help you today?", "#3498db")
   
    def add_message(self, sender, message, color="#333333"):
        """Add a message to the chat display"""
        self.chat_display.config(state=tk.NORMAL)
       
        timestamp = datetime.now().strftime("%H:%M:%S")
       
        # Add sender and timestamp
        self.chat_display.insert(tk.END, f"\n[{timestamp}] {sender}: ", f"sender_{sender}")
        self.chat_display.insert(tk.END, f"{message}\n", "message")
       
        # Configure tags for styling
        self.chat_display.tag_config(f"sender_{sender}", foreground=color, font=('Arial', 10, 'bold'))
        self.chat_display.tag_config("message", foreground="#333333", font=('Arial', 10))
       
        self.chat_display.config(state=tk.DISABLED)
        self.chat_display.see(tk.END)
   
    def get_ai_response(self, user_message):
        """Get response from AI API"""
        try:
            # Add user message to conversation history
            self.conversation_history.append({"role": "user", "content": user_message})
           
            # Keep only last 10 messages to avoid token limits
            if len(self.conversation_history) > 10:
                self.conversation_history = self.conversation_history[-10:]
           
            # Prepare API request
            data = {
                "model": "deepseek/deepseek-r1:free",
                "messages": [
                    {"role": "system", "content": "You are a helpful, friendly, and conversational AI assistant. Provide clear, engaging responses and feel free to ask follow-up questions to keep the conversation flowing."},
                    *self.conversation_history
                ],
                "temperature": 0.7,
                "max_tokens": 500
            }
           
            response = requests.post(
                self.api_url,
                headers=self.headers,
                data=json.dumps(data),
                timeout=30
            )
           
            if response.status_code == 200:
                response_data = response.json()
                ai_message = response_data['choices'][0]['message']['content']
               
                # Add AI response to conversation history
                self.conversation_history.append({"role": "assistant", "content": ai_message})
               
                return ai_message
            else:
                return f"Sorry, I encountered an error: {response.status_code}"
               
        except requests.exceptions.Timeout:
            return "Sorry, the request timed out. Please try again."
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
   
    def send_text_message(self, event=None):
        """Send text message"""
        message = self.text_input.get().strip()
        if not message:
            return
       
        # Add user message to display
        self.add_message("You", message, "#27ae60")
        self.text_input.delete(0, tk.END)
       
        # Update status
        self.status_label.config(text="Getting response...", fg="#f39c12")
        self.root.update()
       
        # Get AI response in a separate thread
        def get_response():
            ai_response = self.get_ai_response(message)
            self.root.after(0, lambda: self.handle_ai_response(ai_response))
       
        threading.Thread(target=get_response, daemon=True).start()
   
    def handle_ai_response(self, response):
        """Handle AI response and speak it"""
        self.add_message("Bot", response, "#3498db")
        self.status_label.config(text="Ready to chat!", fg="#27ae60")
       
        # Speak the response
        self.speak_text(response)
   
    def speak_text(self, text):
        """Convert text to speech"""
        def speak():
            try:
                # Clean text for better speech
                clean_text = re.sub(r'[*_`#]', '', text)  # Remove markdown
                clean_text = re.sub(r'\n+', '. ', clean_text)  # Replace newlines with periods
                self.tts_engine.say(clean_text)
                self.tts_engine.runAndWait()
            except:
                pass
       
        threading.Thread(target=speak, daemon=True).start()
   
    def toggle_voice(self):
        """Toggle voice input"""
        if not self.is_listening:
            self.start_voice_input()
        else:
            self.stop_voice_input()
   
    def start_voice_input(self):
        """Start voice input"""
        self.is_listening = True
        self.voice_btn.config(text="🔴 Stop Voice", bg="#e74c3c")
        self.status_label.config(text="Listening... Speak now!", fg="#e74c3c")
       
        def listen():
            try:
                with self.microphone as source:
                    self.recognizer.adjust_for_ambient_noise(source)
               
                while self.is_listening:
                    try:
                        with self.microphone as source:
                            audio = self.recognizer.listen(source, timeout=1, phrase_time_limit=5)
                       
                        text = self.recognizer.recognize_google(audio)
                        if text:
                            self.root.after(0, lambda: self.handle_voice_input(text))
                    except sr.WaitTimeoutError:
                        continue
                    except sr.UnknownValueError:
                        continue
                    except Exception as e:
                        self.root.after(0, lambda: self.status_label.config(text=f"Voice error: {str(e)}", fg="#e74c3c"))
                        break
            except Exception as e:
                self.root.after(0, lambda: self.status_label.config(text=f"Microphone error: {str(e)}", fg="#e74c3c"))
       
        threading.Thread(target=listen, daemon=True).start()
   
    def stop_voice_input(self):
        """Stop voice input"""
        self.is_listening = False
        self.voice_btn.config(text="🎤 Start Voice", bg="#27ae60")
        self.status_label.config(text="Voice input stopped", fg="#95a5a6")
   
    def handle_voice_input(self, text):
        """Handle voice input"""
        self.text_input.delete(0, tk.END)
        self.text_input.insert(0, text)
        self.send_text_message()
   
    def clear_chat(self):
        """Clear chat history"""
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.delete(1.0, tk.END)
        self.chat_display.config(state=tk.DISABLED)
        self.conversation_history.clear()
        self.add_message("Bot", "Chat cleared! How can I help you?", "#3498db")
   
    def run(self):
        """Start the application"""
        try:
            self.root.mainloop()
        except KeyboardInterrupt:
            self.stop_voice_input()

# Simple command-line version
class SimpleChatBot:
    def __init__(self):
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
        self.api_key = "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"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        self.conversation_history = []
   
    def get_response(self, message):
        """Get AI response"""
        self.conversation_history.append({"role": "user", "content": message})
       
        data = {
            "model": "deepseek/deepseek-r1:free",
            "messages": [
                {"role": "system", "content": "You are a helpful AI assistant."},
                *self.conversation_history[-10:]  # Keep last 10 messages
            ]
        }
       
        try:
            response = requests.post(self.api_url, headers=self.headers, data=json.dumps(data))
            if response.status_code == 200:
                ai_message = response.json()['choices'][0]['message']['content']
                self.conversation_history.append({"role": "assistant", "content": ai_message})
                return ai_message
            else:
                return f"Error: {response.status_code}"
        except Exception as e:
            return f"Error: {str(e)}"
   
    def run(self):
        """Run simple chat"""
        print("🤖 Simple ChatBot started! Type 'quit' to exit.")
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() in ['quit', 'exit']:
                break
            if user_input:
                response = self.get_response(user_input)
                print(f"Bot: {response}")

if __name__ == "__main__":
    print("Choose mode:")
    print("1. GUI ChatBot with Voice (Recommended)")
    print("2. Simple Command Line ChatBot")
   
    choice = input("Enter choice (1 or 2): ").strip()
   
    if choice == "2":
        bot = SimpleChatBot()
        bot.run()
    else:
        try:
            bot = InteractiveChatBot()
            bot.run()
        except ImportError as e:
            print(f"GUI dependencies missing: {e}")
            print("Installing required packages:")
            print("pip install speechrecognition pyttsx3 pyaudio")
            print("\nRunning simple version instead...")
            bot = SimpleChatBot()
            bot.run()

Choose mode:
1. GUI ChatBot with Voice (Recommended)
2. Simple Command Line ChatBot


Enter choice (1 or 2):  1
